### Steps
1. Write a function to retrieve all the postal codes in Singapore, preferably from a REST API.

2. Based on each postal code, call a REST API to fetch the lat and lng for each postal code

3. Put the results of (1) and (2) into a dataframe with the following columns:
<br>
PostaL_Code   |   Lat   |   Lng


4. Ask the user to input a reference postal code. For each row in the dataframe, calculate the distance of each postal code from the reference point. Add the result into the dataframe. The dataframe now looks like this:

<br>
PostaL_Code | Lat | Lng |Distance_from_ref_point




In [2]:
import pandas as pd
import json
import requests
import http.client, urllib.parse
from math import radians, cos, sin, asin, sqrt


In [27]:
def load_all_postal_codes:
    data=pd.read_csv('SG.txt',sep="\t",header=None)
    data=data.dropna(axis=1).drop(columns=0)
    data.columns=['Postal Code','Address','Latitude','Longtitude']
    return data

,Postal Code,Address,Latitude,Longtitude
0,18906,Straits Boulevard,1.2758,103.8496
1,18907,Straits Boulevard,1.2749,103.8517
2,18910,Marina Gardens Drive,1.2796,103.8690
3,18915,Central Boulevard,1.2737,103.8601
4,18916,Central Boulevard,1.2798,103.8515
...,...,...,...,...
121149,886129,Upper Changi Road North,1.3523,103.9665
121150,887327,Kranji Loop,1.4336,103.7586
121151,887328,Pasir Panjang Road,1.2772,103.7958
121152,918104,Airport Cargo Road,1.3753,103.9967


In [160]:
conn = http.client.HTTPConnection('geocode.xyz')

params = urllib.parse.urlencode({
    'auth': '191249184319614221290x60675',
    'locate': location,
    'region': 'SG',
    'json': 1,
    })

conn.request('GET', '/?{}'.format(params))

res = conn.getresponse()
read = res.read()

response_dict = json.loads(read)

In [72]:
## code

#load postal codes downloaded from GeoNames Postal Code into memory
#weblink:http://download.geonames.org/export/zip/
def load_all_postal_codes():
    '''load postal codes downloaded from GeoNames Postal Code into memory
    Link:http://download.geonames.org/export/zip/
    '''
    data=pd.read_csv('SG.txt',sep="\t",header=None)
    data=data.dropna(axis=1).drop(columns=0)
    data.columns=['Postal Code','Address','Latitude','Longtitude']
    
    return data

def find_lat_long(code):
    
    conn = http.client.HTTPConnection('geocode.xyz')

    params = urllib.parse.urlencode({
    'auth': '191249184319614221290x60675',
    'locate': code,
    'region': 'SG',
    'json': 1,
    })

    conn.request('GET', '/?{}'.format(params))

    res = conn.getresponse()
    read = res.read()

    response_dict = json.loads(read)
    
    return code,response_dict['latt'],response_dict['longt']

def prompt_user():
    while True:
        ref=input(' Please key in your desired reference postal code ')
        if len(ref)==6 and ref.isnumeric():
            break

            
    return ref

#link:https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points 
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [77]:
df=load_all_postal_codes()

reference=prompt_user()

postal_code,lat,long=find_lat_long(reference)

df['distance'] = df.apply(lambda x: round(haversine(x['Longtitude'],x['Latitude'],float(long),float(lat)),2), axis=1)
print(df)

 Please key in your desired reference postal code 530440
        Postal Code                  Address  Latitude  Longtitude  distance
0             18906        Straits Boulevard    1.2758    103.8496     12.48
1             18907        Straits Boulevard    1.2749    103.8517     12.49
2             18910     Marina Gardens Drive    1.2796    103.8690     11.40
3             18915        Central Boulevard    1.2737    103.8601     12.30
4             18916        Central Boulevard    1.2798    103.8515     11.99
...             ...                      ...       ...         ...       ...
121149       886129  Upper Changi Road North    1.3523    103.9665      8.64
121150       887327              Kranji Loop    1.4336    103.7586     16.18
121151       887328       Pasir Panjang Road    1.2772    103.7958     15.70
121152       918104       Airport Cargo Road    1.3753    103.9967     11.48
121153       918146        Airport Boulevard    1.3550    103.9891     10.96

[121154 rows x 5 c

In [75]:
lat,long,postal_code

('1.35488', '103.88967', '530110')

In [76]:
df[df['Postal Code']==530440]

,Postal Code,Address,Latitude,Longtitude,distance
65649,530440,Hougang Avenue 8,1.3791,103.8935,2.73


In [54]:
df

,Postal Code,Address,Latitude,Longtitude,distance
0,18906,Straits Boulevard,1.2758,103.8496,0.0
1,18907,Straits Boulevard,1.2749,103.8517,0.0
2,18910,Marina Gardens Drive,1.2796,103.8690,0.0
3,18915,Central Boulevard,1.2737,103.8601,0.0
4,18916,Central Boulevard,1.2798,103.8515,0.0
...,...,...,...,...,...
121149,886129,Upper Changi Road North,1.3523,103.9665,0.0
121150,887327,Kranji Loop,1.4336,103.7586,0.0
121151,887328,Pasir Panjang Road,1.2772,103.7958,0.0
121152,918104,Airport Cargo Road,1.3753,103.9967,0.0


In [53]:
haversine(103.8935,1.3791,103.8935,1.3791)

0.0

In [68]:
df['distance'] = df.apply(lambda x: round(haversine(x['Longtitude'],x['Latitude'],float(long),float(lat)),2), axis=1)

In [69]:
df[df['Postal Code']==530440]

,Postal Code,Address,Latitude,Longtitude,distance
65649,530440,Hougang Avenue 8,1.3791,103.8935,0.0


In [65]:
haversine

<function __main__.haversine(lon1, lat1, lon2, lat2)>